In [85]:
import pandas as pd
import wordPreprocessing as wpp
import countryExtractor as ce
import time
import sentimentAnalyser as sa

In [77]:
data = pd.read_csv('../data/new_data_vdss.tsv.xz', sep='\t', compression='xz')

In [78]:
# Datum bereinigen & redundante Spalten löschen
data['datum'] = data['pubtime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
data['datum'] = pd.to_datetime(data['datum'])
del data['pubtime']
del data['medium_code']
del data['regional']
del data['doctype']
del data['language']
del data['char_count']
del data['dateline']
del data['subhead']
del data['content_id']

In [84]:
data

,id,medium_name,rubric,doctype_description,head,content,datum
0,47612274,blick.ch,Wirtschaft,Online medium,Schweden beschliesst starke Leitzinserhöhung,<tx><ld><p>Die schwedische Notenbank stemmt si...,2022-09-20
1,46153316,srf.ch,Sport - Mehr Sport,Online medium,Handball-Nati: Schweizerinnen unterliegen Pole...,<tx><ld><p>Die Schweizer Handball-Nati der Fra...,2022-04-24
2,45099545,swissinfo.ch,Gesundheit,Online medium,"""Wir befürchten, dass die Tabakinitiative die ...",<tx><ld><p>Wirtschaftskreise wehren sich gegen...,2022-01-11
3,47381556,NZZ am Sonntag,Mensch und Medizin,Regional weekly newspaper,News,<tx><zt>Medizincannabis und Nikotin gehen Hand...,2022-08-28
4,47256809,srf.ch,Radio SRF 1,Online medium,Wenn Wassersportler fliegen lernen,<tx><ld><p>Man sieht sie bald auf jedem Schwei...,2022-08-15
...,...,...,...,...,...,...,...
153195,48273157,Berner Zeitung,Region,Regional daily newspaper,Warum Opfer von Online-Hetze oft machtlos sind,<tx><ld><p>Am Internet-Pranger Ein linksradika...,2022-11-24
153196,45361730,20 minuten,St. Gallen/Region,Regional daily newspaper,Bürocontainer in Vollbrand,<tx><p>ARBON. Aus noch unbekannten Gründen bra...,2022-02-07
153197,45122731,blick.ch,Politik,Online medium,Bundespräsident Cassis zu Besuch in Wien,<tx><ld><p>Die nächsten Schritte der Schweiz i...,2022-01-13
153198,45429217,NZZ am Sonntag,Schweiz,Regional weekly newspaper,Classe politique,"<tx><p>Ignazio Cassis, Globetrotter, twitterte...",2022-02-13


In [83]:
# Aufbereitung der blick.ch Rubriken
data.loc[data['medium_name'] == 'blick.ch', 'rubric'] = data.loc[data['medium_name'] == 'blick.ch', 'rubric'].str.split(' - ').str[0]
filtered_data = data[data['medium_name'] == 'blick.ch']['rubric']
data['rubric'].unique()

array(['Wirtschaft', 'Sport - Mehr Sport', 'Gesundheit',
       'Mensch und Medizin', 'Radio SRF 1', 'News - International',
       'Region', 'Ausland', 'News - Schweiz', 'Meinungen', 'Sport',
       'Zürich', 'St. Gallen/Region', 'Leben', 'People',
       'Zürich und Region', 'News', 'International', 'Wochenende',
       'Debatte', 'Wetter', 'Meinung und Debatte', 'Schweiz', 'Mobil',
       'Feuilleton', 'News - Gesellschaft', 'Sport - Fussball',
       'Letzte Seite', 'Panorama', 'Front', 'Services', 'Invest',
       'Sport - Eishockey', 'SonntagsBlick', 'Basel/Region', 'Forum',
       'Krieg in der Ukraine', 'Meteo - Meteo-Stories', 'Video',
       'Luzern/Region', 'Kehrseite', 'Coopzeitung Weekend',
       'Computer und Technik', 'WM 2022', 'Bern/Region', 'Sport - Tennis',
       'Politik', 'Pro Global', 'Sport - Leichtathletik', 'Magazin',
       'Kultur & Gesellschaft', 'Kultur - Film & Serien', 'Konflikte',
       'Influencer Radar', 'Forschung und Technik',
       'News - Absti

In [22]:
# Headlines aufbereiten mit Klasse word_preprocessing

preprocessor = wpp.headlinePreprocessing()

start_time = time.time()
data['content_processed'] = data['content'][:200].apply(lambda x: preprocessor.preprocess_text(x) if isinstance(x, str) else [])
end_time = time.time()

elapsed_time = end_time - start_time

del data['content']

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
Elapsed Time: 57.64 seconds


In [21]:
# Extraktion der Länder
# testweise nur 100 erste Zeilen
extractor = ce.countryExtractor()
data['countries'] = data['content'].apply(lambda x: extractor.get_country(x) if isinstance(x, str) else [])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\linus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\linus\AppData\Local\Temp\ipykernel_18232\1111091668.py", line 3, in <cell line: 3>
    data['laender'] = data['content'].apply(lambda x: extractor.get_country(x) if isinstance(x, str) else [])
  File "C:\Users\linus\anaconda3\lib\site-packages\pandas\core\series.py", line 4631, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "C:\Users\linus\anaconda3\lib\site-packages\pandas\core\apply.py", line 1025, in apply
    return self.apply_standard()
  File "C:\Users\linus\anaconda3\lib\site-packages\pandas\core\apply.py", line 1076, in apply_standard
    mapped = lib.map_infer(
  File "pandas\_libs\lib.pyx", line 2834, in pandas._libs.lib.map_infer
  File "C:\Users\linus\AppData\Local\Temp\ipykernel_18232\1111091668.py", line 3, in <lambda>
 

In [90]:
# Sentiment-Analyse
# testweise nur 100 erste Zeilen
sentiment_analyser = sa.sentimentAnalyser()
data['Sentiment'] = data['content'][0:100].apply(lambda x: sentiment_analyser.get_topic_sentiments(x))